In [12]:
%matplotlib inline
import pandas as pd
import numpy as np
import joblib as jl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import StratifiedShuffleSplit

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

root = 'dl'

# Load raw data

In [13]:
labels = pd.read_csv('../../tmp/1_METONLY_vs_METX/matched_CEM_table.csv', header=0).rename({'Unnamed: 0': 'PIN'}, axis=1)[['PIN', 'CLASS']].set_index('PIN')
labels.head()

,CLASS
PIN,
568483,1
923748,0
2035328,0
2239386,1
2244533,1


In [14]:
data = jl.load(open('../../tmp/item_days_raw_data_.pkl', 'rb')).loc[labels.index, 'seq']
df = pd.DataFrame(columns=['Seq', 'Class'], index=data.index)
df.loc[:, 'Seq'] = data
df.loc[:, 'Class'] = labels['CLASS']
df.head()

,Seq,Class
PIN,,
568483,10990 0 23 16 725 0 10990 0 10990 0 10990 0 10...,1
923748,66653 0 65070 0 66515 5 10962 1 105 1 56807 42...,0
2035328,66536 0 74995 0 66512 0 66551 0 66608 0 66716 ...,0
2239386,10991 0 23 105 73928 0 66560 0 66515 0 66551 0...,1
2244533,10990 0 23 74 23 0 10990 1 73928 0 66500 0 665...,1


# Separate MBS items and timepoints

In [15]:
for idx in df.index:
    _tmp = df.loc[idx, 'Seq'].split(' ')
    df.loc[idx, 'mbs_seq'] = ' '.join(_tmp[::2])
    df.loc[idx, 'times_seq'] = ' '.join(_tmp[1::2])

In [16]:
df.head()

,Seq,Class,mbs_seq,times_seq
PIN,,,,
568483,10990 0 23 16 725 0 10990 0 10990 0 10990 0 10...,1,10990 23 725 10990 10990 10990 10993 23 85311 ...,0 16 0 0 0 0 0 15 0 0 13 0 0 0 22 0 0 0 0 0 0 ...
923748,66653 0 65070 0 66515 5 10962 1 105 1 56807 42...,0,66653 65070 66515 10962 105 56807 10990 23 109...,0 0 5 1 1 42 0 26 35 0 0 0 9 0 0 0 5 0 0 0 21 ...
2035328,66536 0 74995 0 66512 0 66551 0 66608 0 66716 ...,0,66536 74995 66512 66551 66608 66716 66560 7392...,0 0 0 0 0 0 0 9 1 10 9 138 1 3 31 24 63 0 0 0 ...
2239386,10991 0 23 105 73928 0 66560 0 66515 0 66551 0...,1,10991 23 73928 66560 66515 66551 66536 2517 10...,0 105 0 0 0 0 4 2 27 1 1 7 0 18 132 0 0 0 1 0 ...
2244533,10990 0 23 74 23 0 10990 1 73928 0 66500 0 665...,1,10990 23 23 10990 73928 66500 66560 66551 23 1...,0 74 0 1 0 0 0 2 0 87 0 1 0 0 0 0 0 2 0 6 0 8 ...


# Seq2Tokens

In [17]:
# Define tokenizer object
tokenizer = Tokenizer(char_level=False, lower=False, split=' ')

# Fit on corpus and extract tokenized sequences
tokenizer.fit_on_texts(df['mbs_seq'])
n_feat = len(tokenizer.word_index.keys())
print(n_feat)
seq = tokenizer.texts_to_sequences(df['mbs_seq'])

# Pad tokenized sequences
lengths = [len(x) for x in seq]
maxlen = int(np.percentile(lengths, 95))
p_seq = pad_sequences(seq, maxlen=maxlen)
print(p_seq.shape)

2773
(11744, 445)


In [34]:
t_seq = [map(int, df.loc[idx, 'times_seq'].split(' ')) for idx in df.index]
p_t_seq = pad_sequences(t_seq, maxlen=maxlen)
print(p_t_seq.shape)

(11744, 445)


# Training - Validation - Test split

In [35]:
# Full dataset
y = df['Class'].ravel()
X = p_seq
X_t = p_t_seq

# Learn / Test
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=0)
learn_idx, test_idx = next(sss.split(X, y))

X_learn, y_learn = X[learn_idx, :], y[learn_idx]
X_test, y_test = X[test_idx, :], y[test_idx]

X_learn_t = X_t[learn_idx, :]
X_test_t = X_t[test_idx, :]

print('{} learn / {} test'.format(len(y_learn), len(y_test)))

# Training / Validation
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=1)
train_idx, valid_idx = next(sss.split(X_learn, y_learn))

X_train, y_train = X_learn[train_idx, :], y_learn[train_idx]
X_valid, y_valid = X_learn[valid_idx, :], y_learn[valid_idx]

X_train_t = X_learn_t[train_idx, :]
X_valid_t = X_learn_t[valid_idx, :]

print('{} training / {} validation'.format(len(y_train), len(y_valid)))

5872 learn / 5872 test
5284 training / 588 validation


# Init embedding matrix

In [36]:
D = 100
embedding = pd.read_csv('../../tmp/embedding.{}d.csv'.format(D), header=0, index_col=0)
embedding_matrix = embedding.values

# pre-Model: LSTM(timestamps)

In [112]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import CuDNNLSTM, LSTM
from keras.layers import Conv1D
from keras.layers import Flatten
from keras.callbacks import EarlyStopping
from keras.layers import Bidirectional

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(maxlen, 1)))
model.add(LSTM(32, dropout=0.5, recurrent_dropout=0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           (None, 443, 64)           256       
_________________________________________________________________
lstm_11 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 33        
Total params: 12,705
Trainable params: 12,705
Non-trainable params: 0
_________________________________________________________________


In [ ]:
callbacks = [EarlyStopping(monitor='val_loss', patience=15)] 

history = model.fit(X_train_t.reshape(5284, 445, 1), y_train,
                    epochs=100,
                    callbacks=callbacks,
                    batch_size=128,
                    validation_data=(X_valid_t.reshape(588, 445, 1),
                                     y_valid))

Train on 5284 samples, validate on 588 samples
Epoch 1/100
5284/5284 [==============================] - 46s 9ms/step - loss: 0.6772 - acc: 0.5719 - val_loss: 0.6573 - val_acc: 0.6241
Epoch 2/100
5284/5284 [==============================] - 46s 9ms/step - loss: 0.6621 - acc: 0.6103 - val_loss: 0.6651 - val_acc: 0.6088
Epoch 3/100
5284/5284 [==============================] - 46s 9ms/step - loss: 0.6589 - acc: 0.6147 - val_loss: 0.6532 - val_acc: 0.6156
Epoch 4/100
5284/5284 [==============================] - 46s 9ms/step - loss: 0.6499 - acc: 0.6240 - val_loss: 0.6678 - val_acc: 0.6241
Epoch 5/100
5284/5284 [==============================] - 47s 9ms/step - loss: 0.6532 - acc: 0.6221 - val_loss: 0.6540 - val_acc: 0.6361
Epoch 6/100
5284/5284 [==============================] - 46s 9ms/step - loss: 0.6445 - acc: 0.6347 - val_loss: 0.6560 - val_acc: 0.6293
Epoch 7/100
5284/5284 [==============================] - 45s 8ms/step - loss: 0.6475 - acc: 0.6344 - val_loss: 0.6601 - val_acc: 0.6310
E

In [ ]:
plt.figure(dpi=100)
t = history.epoch

plt.subplot(211)
plt.plot(t, history.history['loss'], label='loss', color='C0')
plt.plot(t, history.history['val_loss'], label='val_loss', color='C1')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc=1)

plt.subplot(212)
plt.plot(t, history.history['acc'], label='acc', color='C2')
plt.plot(t, history.history['val_acc'], label='val_acc', color='C3')
plt.ylim([0.5, 1])
plt.ylabel('acc')
plt.xlabel('epochs')
plt.legend(loc=1)

plt.tight_layout()

In [ ]:
test_loss, test_acc = model.evaluate(X_test_t.reshape(5872, 445, 1), y_test)
print('test_loss: {:1.4f} - test_acc: {:1.4f}'.format(test_loss, test_acc))